<a href="https://colab.research.google.com/github/samuel0711/Desafio/blob/main/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
import psycopg2

In [27]:
# Função para criar conexão no banco
def conecta_db():
  con = psycopg2.connect(host='ec2-3-222-74-92.compute-1.amazonaws.com', 
                         database='dfo2vqe9hs2knm',
                         user='zgbodaynfjorzt',
                         port='5432', 
                         password='d2b11c0c6039df67f39d22f680fb92a843a1b652e3d5b692f6670c0a2abc2804')
  return con

In [32]:
#Consulta em tabela específica
def pegar_tabela(tabela):
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  #construindo query
  query = 'SELECT * FROM '+tabela
  cur.execute(query)
  # pegar todos os dados da query e armazenar dentro de uma lista
  data = cur.fetchall()

  registros = []
  for rec in data:
    registros.append(rec)
  con.close()

  #print(registros[0])
  return registros

#Consulta sobre nomes das colunas de uma tabela
def pegar_colunas(tabela):
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  #construindo query
  query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = '"+tabela+"'"
  cur.execute(query)
  col_names = cur.fetchall()
  con.close()

  col_names = [col[0] for col in col_names]
  #print(col_names)
  return col_names

# Função para de consulta ao banco
def consultar_db():
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  # execute uma declaração
  print('Versão do banco de dados PostgreSQL:')
  cur.execute ('SELECT version()')
  # exibir a versão do servidor de banco de dados PostgreSQL
  db_version = cur.fetchone()
  print(db_version)

  #verificando tabelas disponíveis
  cur.execute ("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema'")
  tables = cur.fetchall()
  for rec in tables:
    print(rec)

#Retorna as consultas e monta um dataframe
def fazer_consulta(tabela):
  registros = pegar_tabela(tabela)
  colunas = pegar_colunas(tabela)
  df = pd.DataFrame(registros, columns=colunas)
  return df

In [33]:
consultar_db()

Versão do banco de dados PostgreSQL:
('PostgreSQL 14.4 (Ubuntu 14.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',)
('public', 'forms', 'zgbodaynfjorzt', None, True, False, True, False)
('public', 'order_items', 'zgbodaynfjorzt', None, True, False, True, False)
('public', 'orders', 'zgbodaynfjorzt', None, True, False, True, False)


In [34]:
df = fazer_consulta('orders')

('c677716e0f2901f', '001b7aed7554265e4e5f', datetime.datetime(2022, 3, 22, 21, 10, 49), 'Entregue', 'CARTÃO', Decimal('198.63'), Decimal('0.00'), True, 'Osasco', 'SP', 'BR', 'google')
['id_pedido', 'id_cliente', 'data_pedido', 'status_pagamento', 'tipo_pagamento', 'valor_total', 'valor_desconto', 'frete_gratis', 'cidade', 'estado', 'locale', 'utm_source']


In [35]:
df

,id_pedido,id_cliente,data_pedido,status_pagamento,tipo_pagamento,valor_total,valor_desconto,frete_gratis,cidade,estado,locale,utm_source
0,c677716e0f2901f,001b7aed7554265e4e5f,2022-03-22 21:10:49,Entregue,CARTÃO,198.63,0.00,True,Osasco,SP,BR,google
1,df063dd3f6b1641,d107878c463e7c969ce6,2022-03-22 21:47:08,Entregue,CARTÃO,198.63,0.00,True,São Paulo,SP,BR,influencer
2,dd69f88884e45a8,44845cf79366dbcecf00,2022-03-08 22:50:22,Entregue,CARTÃO,260.96,0.00,True,Bauru,SP,BR,facebook
3,2c556256ec3a77e,2aa1a61cfa7be6c51167,2022-01-21 14:53:44,Entregue,CARTÃO,225.24,0.00,False,Ananindeua,PA,BR,facebook
4,0262be22fc1c6f0,10c138383539f471831f,2022-03-08 22:50:24,Entregue,CARTÃO,199.31,0.00,True,Saquarema,RJ,BR,facebook
...,...,...,...,...,...,...,...,...,...,...,...,...
55073,efb2662a964fc3f,2b24e80a5c68b1e2beea,2022-05-27 20:49:02,Entregue,CARTÃO,106.59,164.82,True,Barueri,SP,BR,None
55074,eb5927c83f6a8a3,46341dc64e4be47e5a6b,2022-05-31 19:56:57,Em transporte,CARTÃO,93.27,178.55,True,Manaus,AM,BR,None
55075,9e1899f8424623c,c24793059896d1ea37c6,2022-06-02 18:14:21,Pagamento aprovado,CARTÃO,133.23,137.35,True,São Paulo,SP,BR,None
55076,57e2e13e080c377,f36d044ba5ba50335679,2022-06-05 10:27:14,Cancelado,PIX,109.88,164.82,True,Santos,SP,BR,None
